In [1]:
import numpy as np
from wave_monitor import proto

t = np.linspace(0, 1, 1_000_001)
y = np.sin(2 * np.pi * 440 * t)
print(f"{y.nbytes:,}")
print(f"{len(proto.encode(y)):,}")
y.dtype


8,000,008
8,000,056


dtype('float64')

In [2]:
y32 = y.astype(np.float32)
y16 = y.astype(np.float16)

def to_int16(y):
    y = np.clip(y, -1, 1)
    return (y * np.iinfo(np.int16).max).astype(np.int16)

def from_int16(y):
    return y.astype(np.float32) / np.iinfo(np.int16).max

yint = to_int16(y)
print(f"float32, {y32.nbytes:,} {len(proto.encode(y32)):,}")
print(f"float16, {y16.nbytes:,} {len(proto.encode(y16)):,}")
print(f"int16, {yint.nbytes:,} {len(proto.encode(yint)):,}")

float32, 4,000,004 4,000,052
float16, 2,000,002 2,000,050
int16, 2,000,002 2,000,050


In [3]:
print("Original float64")
%timeit -r 5 -n 100 y
%timeit -r 5 -n 100 proto.encode(y)

print("float32")
%timeit -r 5 -n 100 y.astype(np.float32)
%timeit -r 5 -n 100 proto.encode(y.astype(np.float32))

print("float16")
%timeit -r 5 -n 100 y.astype(np.float16)
%timeit -r 5 -n 100 proto.encode(y.astype(np.float16))

print("int16")
%timeit -r 5 -n 100 to_int16(y)
%timeit -r 5 -n 100 proto.encode(to_int16(y))

Original float64
10 ns ± 1.79 ns per loop (mean ± std. dev. of 5 runs, 100 loops each)
3.5 ms ± 345 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)
float32
1 ms ± 44.7 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)
3.01 ms ± 60.6 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)
float16
2.15 ms ± 35.4 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)
2.94 ms ± 31.2 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)
int16
5.18 ms ± 113 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)
5.91 ms ± 64.3 μs per loop (mean ± std. dev. of 5 runs, 100 loops each)


In [8]:
np.abs(y32 - y).max(), np.abs(y16 - y).max(), np.abs(from_int16(yint) - y).max()

(np.float64(2.9798668754743574e-08),
 np.float64(0.00024413489863339688),
 np.float64(3.052832017702567e-05))

In [ ]:
a = np.zeros(2)
a, a.astype(np.float32), a.astype(np.float16), from_int16(to_int16(a))

(array([0., 0.]),
 array([0., 0.], dtype=float32),
 array([0., 0.], dtype=float16),
 array([0., 0.], dtype=float32))

In [11]:
a = np.ones(2) * 0.5
a, a.astype(np.float32), a.astype(np.float16), from_int16(to_int16(a))

(array([0.5, 0.5]),
 array([0.5, 0.5], dtype=float32),
 array([0.5, 0.5], dtype=float16),
 array([0.49998474, 0.49998474], dtype=float32))